In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing
from sklearn.decomposition import PCA
import os

In [2]:
cobbles = pd.read_excel("Found cobble intervals in March-April through signals.xlsx")
cobbles

,Unnamed: 0,Start Time,End Time,Duration (minutes)
0,0,2024-02-15 00:39:40.490,2024-02-15 01:18:00.490,38.333333
1,1,2024-02-15 01:38:01.510,2024-02-15 02:00:57.510,22.933333
2,2,2024-02-15 02:04:55.510,2024-02-15 02:22:39.510,17.733333
3,3,2024-02-15 02:26:38.540,2024-02-15 03:08:08.540,41.500000
4,4,2024-02-15 14:33:32.850,2024-02-15 15:19:33.850,46.016667
...,...,...,...,...
89,89,2024-04-20 10:22:28.680,2024-04-20 11:04:49.720,42.350667
90,90,2024-04-20 12:49:27.810,2024-04-20 13:18:25.810,28.966667
91,91,2024-04-20 15:03:39.890,2024-04-20 15:13:05.890,9.433333
92,92,2024-04-20 20:37:37.150,2024-04-20 21:12:46.150,35.150000


In [4]:
df = pd.read_pickle("mat_absence_and_ghost_rolling_removed.pkl")
df.head()

,[1:0],[1:1],[1:2],[1:3],[1:4],[1:5],[1:6],[1:7],[1:8],[1:9],...,[26.116],[26.117],[26.118],[26.119],[26.120],[26.121],[26.122],[26.123],[13.208],[1.245]
Time,,,,,,,,,,,,,,,,,,,,,
2024-02-15 00:26:05.490,-10092,-16379,13233,10794,-4.153270e+34,-1.622420e+32,-12,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN
2024-02-15 00:26:06.490,-10092,-16379,13233,15672,-6.489480e+32,3.789820e-37,3,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN
2024-02-15 00:26:07.490,-10092,-16379,13233,15672,-1.655980e+35,1.789910e-15,39,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN
2024-02-15 00:26:08.490,-10092,-16379,18111,15672,1.588990e-30,2.466240e-32,11,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN
2024-02-15 00:26:09.490,-10092,-16379,18111,15672,1.407160e+14,-4.046020e+31,-13,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN


In [7]:
#removing cobbles from cleaned-data (mat absence and ghost rolled removed)

def remove_cobbles(df, cobbles):
    cobbles['Start Time'] = pd.to_datetime(cobbles['Start Time'])
    cobbles['End Time'] = pd.to_datetime(cobbles['End Time'])
    
    # Create a boolean mask to identify rows to remove
    mask = pd.Series(False, index=df.index)
    for _, row in cobbles.iterrows():
        start_time = row['Start Time']
        end_time = row['End Time']
        mask |= (df.index >= start_time) & (df.index <= end_time)
    
    # Invert the mask to keep rows not in the mask
    df = df[~mask]
    return df

print("Before cobble removal:", df.shape)
df = remove_cobbles(df, cobbles)
print("After cobble removal:", df.shape)


Before cobble removal: (704134, 2663)
After cobble removal: (701302, 2663)


In [8]:
df_filtered1 = df[(df.index >= datetime(2024, 2, 15)) & (df.index <= datetime(2024, 3, 26))]

In [9]:
df_filtered1.head()

,[1:0],[1:1],[1:2],[1:3],[1:4],[1:5],[1:6],[1:7],[1:8],[1:9],...,[26.116],[26.117],[26.118],[26.119],[26.120],[26.121],[26.122],[26.123],[13.208],[1.245]
Time,,,,,,,,,,,,,,,,,,,,,
2024-02-15 00:26:05.490,-10092,-16379,13233,10794,-4.153270e+34,-1.622420e+32,-12,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN
2024-02-15 00:26:06.490,-10092,-16379,13233,15672,-6.489480e+32,3.789820e-37,3,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN
2024-02-15 00:26:07.490,-10092,-16379,13233,15672,-1.655980e+35,1.789910e-15,39,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN
2024-02-15 00:26:08.490,-10092,-16379,18111,15672,1.588990e-30,2.466240e-32,11,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN
2024-02-15 00:26:09.490,-10092,-16379,18111,15672,1.407160e+14,-4.046020e+31,-13,60.0,60.0,60,...,0,0,1,1,1,0,0,0,NaN,NaN


In [10]:
df_filtered1.tail()

,[1:0],[1:1],[1:2],[1:3],[1:4],[1:5],[1:6],[1:7],[1:8],[1:9],...,[26.116],[26.117],[26.118],[26.119],[26.120],[26.121],[26.122],[26.123],[13.208],[1.245]
Time,,,,,,,,,,,,,,,,,,,,,
2024-03-25 23:58:14.100,7040,-15840,-19405,-17188,2.483320e-32,3.853150e-34,8,-1.60010,0.800049,0,...,0,0,1,1,1,0,0,0,0.0,NaN
2024-03-25 23:58:15.100,7040,-15840,-14973,-17188,6.510470e-27,9.635530e-35,7,-1.30005,0.699951,0,...,0,0,1,1,1,0,0,0,0.0,NaN
2024-03-25 23:58:16.100,7040,-15840,-14973,-17188,NaN,9.477140e-38,2,-0.50000,0.199951,0,...,0,0,1,1,1,0,0,0,0.0,NaN
2024-03-25 23:58:17.100,7040,-15840,-14973,-12757,1.287310e-04,1.543430e-33,9,-0.50000,0.899902,0,...,0,0,1,1,1,0,0,0,0.0,NaN
2024-03-25 23:58:18.100,7040,-15840,-14973,-12757,-4.139960e+34,6.827750e-21,30,3.69995,3.000000,0,...,0,0,1,1,1,0,0,0,0.0,NaN


In [11]:
df_filtered1.to_pickle("training_clean.pkl")

In [12]:
df_filtered2 = df[(df.index >= datetime(2024, 3, 25)) & (df.index <= datetime(2024, 4, 21))]

In [13]:
df_filtered2.head()

,[1:0],[1:1],[1:2],[1:3],[1:4],[1:5],[1:6],[1:7],[1:8],[1:9],...,[26.116],[26.117],[26.118],[26.119],[26.120],[26.121],[26.122],[26.123],[13.208],[1.245]
Time,,,,,,,,,,,,,,,,,,,,,
2024-03-25 01:39:53.730,3176,-29556,-32556,-30371,NaN,9.147540e-41,0,-1.5000,0.000000,0,...,0,0,1,1,1,0,0,0,0.0,NaN
2024-03-25 01:39:54.730,3176,-29556,-32556,-30371,9.700470e-35,-4.138060e+34,-8,-1.6001,-0.800049,0,...,0,0,1,1,1,0,0,0,0.0,NaN
2024-03-25 01:39:55.730,3176,-29556,-32556,-30371,-1.547070e+26,2.367850e-38,1,-1.6001,0.100098,0,...,0,0,1,1,1,0,0,0,0.0,NaN
2024-03-25 01:39:56.730,3176,-29556,-32556,-30371,6.062800e-36,-2.648270e+36,-5,-1.6001,-0.500000,0,...,0,0,1,1,1,0,0,0,0.0,NaN
2024-03-25 01:39:57.730,-28085,-29556,-32556,-30371,3.789360e-37,NaN,-2,-1.5000,-0.199951,0,...,0,0,1,1,1,0,0,0,0.0,NaN


In [14]:
df_filtered2.tail()

,[1:0],[1:1],[1:2],[1:3],[1:4],[1:5],[1:6],[1:7],[1:8],[1:9],...,[26.116],[26.117],[26.118],[26.119],[26.120],[26.121],[26.122],[26.123],[13.208],[1.245]
Time,,,,,,,,,,,,,,,,,,,,,
2024-04-20 23:59:55.290,24028,17315,13814,16232,-1.407330e+14,6.203410e-33,10,-0.300049,1.00000,95,...,0,0,1,1,1,0,0,0,0.0,0.0
2024-04-20 23:59:56.290,24028,17315,13814,25380,2.426150e-35,-2.648310e+36,-5,-0.199951,-0.50000,95,...,0,0,1,1,1,0,0,0,0.0,0.0
2024-04-20 23:59:57.290,24028,17315,13814,25380,2.410130e-35,1.694380e-21,29,3.699950,2.89990,95,...,0,0,1,1,1,0,0,0,0.0,0.0
2024-04-20 23:59:58.290,24028,17315,27794,25380,1.552120e-33,3.228590e-42,0,-1.500000,0.00000,95,...,0,0,1,1,1,0,0,0,0.0,0.0
2024-04-20 23:59:59.290,24028,17315,27794,25380,-1.584220e+29,3.973190e-31,13,-1.500000,1.30005,95,...,0,0,1,1,1,0,0,0,0.0,0.0


In [15]:
df_filtered2.to_pickle("testing_clean.pkl")

In [17]:
#cobble free training...
from datetime import timedelta  # Import timedelta
df_filtered1_1 = df_filtered1
cobbles['Start Time'] = pd.to_datetime(cobbles['Start Time'])
cobbles['End Time'] = pd.to_datetime(cobbles['End Time'])

# Remove rows from 'data' where timestamps fall between 'Start Time' and 'End Time' of stoppages
for index, row in cobbles.iterrows():
    start_time = row['Start Time'] - timedelta(minutes=20)
    end_time = row['Start Time'] - timedelta(seconds=1)
    df_filtered1_1 = df_filtered1_1[~((df_filtered1_1.index >= start_time) & (df_filtered1_1.index <= end_time))]
df_filtered1_1.shape
df_filtered1_1.to_pickle("training_cobble_free.pkl")

In [18]:
df_filtered1_1.shape

(536632, 2663)